In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
### Make sure tf-nightly-gpu (>=1.11) should be installed if you only have TF < 1.11
import tensorflow as tf
import tensorflow as tf; print(tf.__version__)

1.12.0-dev20180922


In [3]:
# Keras
from tensorflow import keras

### 0. Data & Model preparation

In [4]:
# My Data
(x_train,y_train),(x_test,y_test) = keras.datasets.mnist.load_data()
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(60000, 28, 28) (10000, 28, 28) (60000,) (10000,)


In [5]:
# Data usually have X in float32 & Y in int32
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

In [6]:
nClass = 10
nData_train = x_train.shape[0]
nData_test = x_test.shape[0]
nDimIn = x_train.shape[1]*x_train.shape[2]
nDimOut = nClass
print(nData_train, nData_test, nDimIn, nDimOut) 

60000 10000 784 10


In [7]:
# My Keras Model
def myModel():
    dataIn = keras.Input(shape=(nDimIn,), name='X')
    fc1 = keras.layers.Dense(40, activation='relu', name='fc1')(dataIn)
    fc2 = keras.layers.Dense(40, activation='relu', name='fc2')(fc1)
    dataOut = keras.layers.Dense(nDimOut, activation='softmax', name='dataOut')(fc2)
    model = keras.Model(inputs=dataIn, outputs=dataOut, name='Y')
    return model
batchSize = 64
nEpoch = 5
nSteps = (int)(nData_train/batchSize)

In [8]:
nStep_train = (int)(nData_train/batchSize)
nStep_test = (int)(nData_test/batchSize)

In [9]:
model = myModel()
model.summary()
keras.backend.clear_session()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
X (InputLayer)               (None, 784)               0         
_________________________________________________________________
fc1 (Dense)                  (None, 40)                31400     
_________________________________________________________________
fc2 (Dense)                  (None, 40)                1640      
_________________________________________________________________
dataOut (Dense)              (None, 10)                410       
Total params: 33,450
Trainable params: 33,450
Non-trainable params: 0
_________________________________________________________________


### 1. Using Keras with tf.data from numpy data on the memory

In [10]:
# Data preprocessing
def myMapFunc_npy(xx, yy):
    norm = tf.constant(255, dtype=tf.float32, shape=(nDimIn,))
    xx = tf.div(tf.reshape(xx, [-1]), norm)
    yy = tf.one_hot(yy, nClass)
    return xx, yy

In [11]:
# prepare a tf.data.dataset
data_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
data_train = data_train.shuffle(nData_train).map(myMapFunc_npy).batch(batchSize).repeat()
data_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))
data_test = data_test.shuffle(nData_test).map(myMapFunc_npy).batch(batchSize).repeat()

In [12]:
model = myModel()
model.compile(optimizer=tf.train.AdamOptimizer(0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [13]:
model.fit(data_train, epochs=nEpoch, validation_data=data_test, steps_per_epoch=nStep_train, validation_steps=nStep_test)

Epoch 1/5
937/937 [==============================] - 4s 4ms/step - loss: 0.3703 - acc: 0.8933 - val_loss: 0.2074 - val_acc: 0.9372
Epoch 2/5
937/937 [==============================] - 4s 4ms/step - loss: 0.1757 - acc: 0.9486 - val_loss: 0.1624 - val_acc: 0.9520
Epoch 3/5
937/937 [==============================] - 3s 4ms/step - loss: 0.1337 - acc: 0.9603 - val_loss: 0.1414 - val_acc: 0.9570
Epoch 4/5
937/937 [==============================] - 4s 4ms/step - loss: 0.1099 - acc: 0.9673 - val_loss: 0.1238 - val_acc: 0.9629
Epoch 5/5
937/937 [==============================] - 4s 4ms/step - loss: 0.0924 - acc: 0.9721 - val_loss: 0.1092 - val_acc: 0.9656


In [14]:
del data_train, data_test, model
keras.backend.clear_session()

### 2. Using Keras with tf.data from TFRecord files

#### 2-1. Write a TFRecord file

In [15]:
def _int64_feature(value):
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
def _float_feature(array):
  return tf.train.Feature(float_list=tf.train.FloatList(value=array))
def _bytes_feature(value):
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def myWriteTFRecord(filename, xx, yy):
    writer = tf.python_io.TFRecordWriter(filename)
    for ii in range(len(yy)):
        myFeat = tf.train.Features(feature={
                    'X': _float_feature(xx[ii]),
                    'Y': _int64_feature(yy[ii])})
        example = tf.train.Example(features=myFeat)
        writer.write(example.SerializeToString())
    writer.close()

In [16]:
# Data should have 2-dim, (N*D)
x_train_vec = x_train.reshape([nData_train,-1])
x_test_vec = x_test.reshape([nData_test,-1])

In [17]:
myWriteTFRecord('mnist_train.tfrecords', x_train_vec, y_train)
myWriteTFRecord('mnist_test.tfrecords', x_test_vec, y_test)

#### 2-2. Training from the TFRecord files

In [18]:
# Data preprocessing
def myMapFunc_onehot(example):
    feature_def = {'X': tf.FixedLenFeature(nDimIn, tf.float32),
                   'Y': tf.FixedLenFeature(1, tf.int64)}
    features = tf.parse_single_example(example, feature_def)
    norm = tf.constant(255, dtype=tf.float32, shape=(nDimIn,))
    xx = tf.div(features['X'], norm)
    yy = tf.reshape(tf.one_hot(features['Y'], nClass, dtype=tf.float32), [-1])
    return xx, yy

In [19]:
# prepare a tf.data.dataset
data_train = tf.data.TFRecordDataset('mnist_train.tfrecords')
data_train = data_train.shuffle(nData_train).map(myMapFunc_onehot).batch(batchSize).repeat()
data_test = tf.data.TFRecordDataset('mnist_test.tfrecords')
data_test = data_test.shuffle(nData_test).map(myMapFunc_onehot).batch(batchSize).repeat()

In [20]:
model = myModel()
model.compile(optimizer=tf.train.AdamOptimizer(0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [21]:
model.fit(data_train, epochs=nEpoch, validation_data=data_test, steps_per_epoch=nStep_train, validation_steps=nStep_test)

Epoch 1/5
937/937 [==============================] - 4s 5ms/step - loss: 0.3756 - acc: 0.8916 - val_loss: 0.1961 - val_acc: 0.9409
Epoch 2/5
937/937 [==============================] - 4s 4ms/step - loss: 0.1727 - acc: 0.9500 - val_loss: 0.1474 - val_acc: 0.9561
Epoch 3/5
937/937 [==============================] - 4s 4ms/step - loss: 0.1320 - acc: 0.9615 - val_loss: 0.1308 - val_acc: 0.9610
Epoch 4/5
937/937 [==============================] - 4s 4ms/step - loss: 0.1094 - acc: 0.9670 - val_loss: 0.1173 - val_acc: 0.9638
Epoch 5/5
937/937 [==============================] - 4s 4ms/step - loss: 0.0943 - acc: 0.9713 - val_loss: 0.1140 - val_acc: 0.9664


In [22]:
del data_train, data_test, model
keras.backend.clear_session()

### 3. Using a pre-made estimator with tf.data from TFRecord files

In [23]:
print(nStep_train, nStep_test)

937 156


In [24]:
# Data preprocessing
def myMapFunc_scalar(example):
    feature_def = {'X': tf.FixedLenFeature(nDimIn, tf.float32),
                   'Y': tf.FixedLenFeature(1, tf.int64)}
    features = tf.parse_single_example(example, feature_def)
    norm = tf.constant(255, dtype=tf.float32, shape=(nDimIn,))
    xx = tf.div(features['X'], norm)
    yy = features['Y']
    return {'X': xx}, yy

In [25]:
def myInputFunc_scalar(filename,numData):
    data_temp = tf.data.TFRecordDataset(filename)
    data_temp = data_temp.shuffle(buffer_size=numData).map(myMapFunc_scalar).batch(batchSize).repeat()
    return data_temp

In [26]:
feature_columns = [tf.feature_column.numeric_column('X', shape=[784,])]
estimator = tf.estimator.DNNClassifier(
 feature_columns=feature_columns,
 hidden_units=[40, 40],
 optimizer=tf.train.AdamOptimizer(0.001),
 n_classes=10,
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp2mwkmeza', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8afe7ea278>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [27]:
estimator.train(input_fn=lambda:myInputFunc_scalar('mnist_train.tfrecords',nData_train), steps=nStep_train*nEpoch)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmp2mwkmeza/model.ckpt.
INFO:tensorflow:loss = 153.49512, step = 0
INFO:tensorflow:global_step/sec: 273.049
INFO:tensorflow:loss = 46.73066, step = 100 (0.367 sec)
INFO:tensorflow:global_step/sec: 302.738
INFO:tensorflow:loss = 18.384815, step = 200 (0.330 sec)
INFO:tensorflow:global_step/sec: 295.827
INFO:tensorflow:loss = 32.845802, step = 300 (0.338 sec)
INFO:tensorflow:global_step/sec: 296.888
INFO:tensorflow:loss = 11.968971, step = 400 (0.337 sec)
INFO:tensorflow:global_step/sec: 291.616
INFO:tensorflow:loss = 13.4842415, step = 500 (0.343 sec)
INFO:tensorflow:global_step/sec: 305.706
INFO:tensorflow:loss = 10.849707, step = 600 (0.327 sec)
INFO:tensorflow:global_step/sec: 297.751
INFO:tensorflow:

In [28]:
eval_result_1 = estimator.evaluate(input_fn=lambda:myInputFunc_scalar('mnist_test.tfrecords',nData_test), steps=nStep_test)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-09-23-21:39:50
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp2mwkmeza/model.ckpt-4685
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [15/156]
INFO:tensorflow:Evaluation [30/156]
INFO:tensorflow:Evaluation [45/156]
INFO:tensorflow:Evaluation [60/156]
INFO:tensorflow:Evaluation [75/156]
INFO:tensorflow:Evaluation [90/156]
INFO:tensorflow:Evaluation [105/156]
INFO:tensorflow:Evaluation [120/156]
INFO:tensorflow:Evaluation [135/156]
INFO:tensorflow:Evaluation [150/156]
INFO:tensorflow:Evaluation [156/156]
INFO:tensorflow:Finished evaluation at 2018-09-23-21:39:50
INFO:tensorflow:Saving dict for global step 4685: accuracy = 0.96644634, average_loss = 0.10984011, global_step = 4685, loss = 7.029767
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 4685: /tmp/t

In [29]:
print(eval_result_1)

{'accuracy': 0.96644634, 'average_loss': 0.10984011, 'loss': 7.029767, 'global_step': 4685}


### 4. Using an estimator from a Keras model with tf.data from TFRecord files

In [31]:
# Data preprocessing
def myMapFunc_onehot(example):
    feature_def = {'X': tf.FixedLenFeature(nDimIn, tf.float32),
                   'Y': tf.FixedLenFeature(1, tf.int64)}
    features = tf.parse_single_example(example, feature_def)
    norm = tf.constant(255, dtype=tf.float32, shape=(nDimIn,))
    xx = tf.div(features['X'], norm)
    yy = tf.reshape(tf.one_hot(features['Y'], nClass, dtype=tf.float32), [-1])
    return xx, yy

In [32]:
def myInputFunc_onehot(filename,numData):
    data_temp = tf.data.TFRecordDataset(filename)
    data_temp = data_temp.shuffle(buffer_size=numData).map(myMapFunc_onehot).batch(batchSize).repeat()
    return data_temp

In [33]:
model = myModel()
model.compile(optimizer=tf.train.AdamOptimizer(0.001),loss='categorical_crossentropy',metrics=['accuracy'])
estimator = keras.estimator.model_to_estimator(model)

INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpzbt75jtt', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8817f93a20>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [35]:
estimator.train(input_fn=lambda:myInputFunc_onehot('mnist_train.tfrecords',nData_train), steps=nStep_train*nEpoch)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='/tmp/tmpzbt75jtt/keras/keras_model.ckpt', vars_to_warm_start='.*', var_name_to_vocab_info={}, var_name_to_prev_var_name={})
INFO:tensorflow:Warm-starting from: ('/tmp/tmpzbt75jtt/keras/keras_model.ckpt',)
INFO:tensorflow:Warm-starting variable: fc1/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: fc1/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: fc2/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: fc2/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: dataOut/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: dataOut/bias; prev_var_name: Unchanged
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_o

In [37]:
eval_result_2 = estimator.evaluate(input_fn=lambda:myInputFunc_onehot('mnist_test.tfrecords', nData_test), steps=nStep_test)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-09-23-21:40:41
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpzbt75jtt/model.ckpt-4685
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [15/156]
INFO:tensorflow:Evaluation [30/156]
INFO:tensorflow:Evaluation [45/156]
INFO:tensorflow:Evaluation [60/156]
INFO:tensorflow:Evaluation [75/156]
INFO:tensorflow:Evaluation [90/156]
INFO:tensorflow:Evaluation [105/156]
INFO:tensorflow:Evaluation [120/156]
INFO:tensorflow:Evaluation [135/156]
INFO:tensorflow:Evaluation [150/156]
INFO:tensorflow:Evaluation [156/156]
INFO:tensorflow:Finished evaluation at 2018-09-23-21:40:42
INFO:tensorflow:Saving dict for global step 4685: accuracy = 0.9629407, global_step = 4685, loss = 0.12257116
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 4685: /tmp/tmpzbt75jtt/model.ckpt-4685

In [38]:
print(eval_result_2)

{'accuracy': 0.9629407, 'loss': 0.12257116, 'global_step': 4685}
